# Дерево решений

Дерево решений (Decision Tree) — это один из методов машинного обучения с учителем, который используется и для задач классификации (как в нашем случае с болезнями), и для регрессии.

Если говорить простым языком, то это алгоритм, который принимает решение, задавая последовательность вопросов о характеристиках объекта.

Профессор, если говорить конкретно о нашем датасете с болезнями сердца, то в каждом Листе (Leaf) находится финальная группа пациентов, которые прошли через все фильтры (узлы) и оказались похожими друг на друга по своим характеристикам.

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 1. Загрузка и предобработка данных

In [6]:
df = pd.read_csv('Heart_Disease_and_Hospitals.csv')

# Преобразуем категориальный признак 'gender' в числовой (0 для Female, 1 для Male)
df['gender'] = df['gender'].apply(lambda x: 1 if x == 'Male' else 0)

# Выбираем признаки (X) и целевую переменную (y)
X = df[['age', 'blood_pressure', 'cholesterol', 'bmi', 'glucose_level', 'gender']]
y = df['heart_disease']

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Конвертируем данные в numpy-массивы для удобства работы
train_data = X_train.values
train_labels = y_train.values
test_data = X_test.values
test_labels = y_test.values

# 2. Реализация самописного дерева решений

In [7]:
# Алгоритм Desicion Tree с нуля
# Узел - это 
class Node:
    """Класс для узла дерева (содержит условие и ветви)."""
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака для разделения
        self.t = t  # пороговое значение
        self.true_branch = true_branch  # ветвь для условия "истина"
        self.false_branch = false_branch # ветвь для условия "ложь"

class Leaf:
    """Класс для листа дерева (содержит предсказание)."""
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        # Cчитаем, объектов какого класса в листе больше всего
        classes = {}
        for label in self.labels:
            classes[label] = classes.get(label, 0) + 1
        # Предсказанием будет самый частый класс
        prediction = max(classes, key=classes.get)
        return prediction

def gini(labels):
    """
    (Требование 1) Расчет критерия Джини.
    Критерий Джини - мера неоднородности выборки.
    Мера хаоса в выборке.
    50 больных и 50 здоровых - максимальная неоднородность (Gini = 0.5)
    100 больных - минимальная неоднородность (Gini = 0)
    Чем ниже значение, тем более однородна выборка.
    Значение варьируется от 0 (все объекты одного класса) до 0.5 (объекты равномерно распределены между двумя классами).
    0.5 - максимум для двух классов, для большего числа классов максимум меньше 0.5
    0 - минимум
    Рассчитывается по формуле: Gini = 1 - Σ(p_i)^2, где p_i - доля объектов класса i в выборке.
    """
    # Подсчет количества объектов каждого класса
    _, counts = np.unique(labels, return_counts=True)
    
    # Расчет вероятностей
    probabilities = counts / len(labels)

    # Расчет индекса Джини
    # np.sum - сумма по всем классам
    return 1 - np.sum(probabilities**2)

def gain(left_labels, right_labels, root_gini):
    """
    (Требование 2) Расчет прироста информации.
    Это оценка полезности разделения данных по определенному признаку и порогу.
    То есть тот, который лучше всего уменьшает неопределенность (неоднородность) в данных - уменьшает бардак.
    Прирост информации - насколько уменьшилась неопределенность (неоднородность) после разделения.
    Рассчитывается по формуле: Gain = Gini(родитель) - p * Gini(левый) - (1 - p) * Gini(правый),
    где p - доля объектов, попавших в левую ветвь.
    0 - минимум (нет прироста информации)
    Максимум зависит от начального значения Gini(родитель) и варьируется от 0 до 0.5 для двух классов.
    0.5 - максимум для двух классов, для большего числа классов максимум меньше 0.5
    """
    # Доля левой ветви
    p = float(len(left_labels)) / (len(left_labels) + len(right_labels))
    # Прирост информации
    return root_gini - p * gini(left_labels) + (1 - p) * gini(right_labels)

def split(data, labels, column_index, t):
    """Разбиение данных на две ветви по условию."""
    # : - это все строки
    # column_index - это столбец по которому мы делаем разделение
    left_indices = np.where(data[:, column_index] <= t)
    right_indices = np.where(data[:, column_index] > t)

    return data[left_indices], data[right_indices], labels[left_indices], labels[right_indices]

# Функция поиска идеального вопроса для разделения
def find_best_split(data, labels, min_samples_leaf):
    # Поиск лучшего признака и порога для разделения
    root_gini = gini(labels)
    best_gain = 0
    best_t, best_index = None, None
    # Количество признаков
    n_features = data.shape[1]

    for index in range(n_features):
        # Сортируем уникальные значения признака
        t_values = np.unique(data[:, index])
        
        # Используем середины между соседними значениями как пороги
        thresholds = (t_values[:-1] + t_values[1:]) / 2
        
        # Перебираем все пороги
        for t in thresholds:
            # Разделяем данные
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

            # Проверяем минимальное количество объектов в листе
            if len(true_labels) < min_samples_leaf or len(false_labels) < min_samples_leaf:
                continue
            
            # Вычисляем прирост информации
            current_gain = gain(true_labels, false_labels, root_gini)

            # Обновляем лучший прирост и параметры разделения
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

def build_tree(data, labels, max_depth=5, min_samples_leaf=5, current_depth=0):
    """
    (Требование 3) Рекурсивное построение дерева с критериями останова.
    """
    # Критерий останова 1: Максимальная глубина
    if current_depth >= max_depth:
        return Leaf(data, labels)

    # Поиск лучшего разделения
    gain, t, index = find_best_split(data, labels, min_samples_leaf)

    # Критерий останова 2 (неявный): Нет прироста информации
    if gain == 0 or index is None:
        return Leaf(data, labels)
    
    # Разделяем данные по лучшему признаку и порогу
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    # Рекурсивно строим ветви
    true_branch = build_tree(true_data, true_labels, max_depth, min_samples_leaf, current_depth + 1)
    false_branch = build_tree(false_data, false_labels, max_depth, min_samples_leaf, current_depth + 1)

    # Возвращаем узел с условием
    return Node(index, t, true_branch, false_branch)

def classify_object(obj, node):
    """Классификация одного объекта с помощью построенного дерева."""
    # Если достигнут лист, возвращаем предсказание
    if isinstance(node, Leaf):
        return node.prediction
    # Иначе переходим по ветвям в зависимости от условия
    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

def predict(data, tree):
    """Получение предсказаний для набора данных."""
    # Классифицируем каждый объект в данных
    return [classify_object(obj, tree) for obj in data]


def accuracy_metric(actual, predicted):
    """
    (Требование 4) Функция подсчета метрики (точности).
    """
    # Подсчет количества правильных предсказаний
    correct_predictions = np.sum(np.array(actual) == np.array(predicted))
    # Вычисление точности в процентах
    return correct_predictions / len(actual) * 100.0

# 3. Обучение, оценка и сравнение моделей

In [8]:
# (Требование 5)
print("--- Обучение и оценка моделей ---")

# Обучение и оценка самописного дерева
# Гиперпараметры (max_depth и min_samples_leaf) можно менять
# max_depth=3, min_samples_leaf=5 подобраны для баланса между переобучением и недообучением
# max_depth - максимальная глубина дерева
# min_samples_leaf - минимальное количество объектов в листе
my_tree = build_tree(train_data, train_labels, max_depth=3, min_samples_leaf=5)
my_predictions = predict(test_data, my_tree)
my_accuracy = accuracy_metric(test_labels, my_predictions)
print("Точность самописного дерева: {:.2f}%".format(my_accuracy))

# Обучение и оценка дерева из sklearn
# Используем те же гиперпараметры для корректного сравнения
# random_state=42 для воспроизводимости результатов
# DecisionTreeClassifier - класс из sklearn для создания дерева решений
sk_tree = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5, random_state=42)
sk_tree.fit(train_data, train_labels)
sk_predictions = sk_tree.predict(test_data)
sk_accuracy = accuracy_score(test_labels, sk_predictions) * 100
print("Точность дерева из sklearn: {:.2f}%".format(sk_accuracy))

# Сравнение предсказаний
print("\n--- Сравнение результатов ---")
if np.array_equal(my_predictions, sk_predictions):
    print("✅ Результаты полностью совпадают!")
else:
    print("❌ Результаты не совпадают.")
    print("Примечание: Небольшие расхождения возможны из-за различий в реализации внутренних алгоритмов (например, при обработке признаков с одинаковым приростом информации).")

--- Обучение и оценка моделей ---
Точность самописного дерева: 50.23%
Точность дерева из sklearn: 88.23%

--- Сравнение результатов ---
❌ Результаты не совпадают.
Примечание: Небольшие расхождения возможны из-за различий в реализации внутренних алгоритмов (например, при обработке признаков с одинаковым приростом информации).
